In [422]:
import csv
import random as rd
from copy import deepcopy

In [423]:
# Chaque item correspond à la place disponible dans chaque projet
projetName = ["projet coolos", "projet claqué", "projet 3", "projet 4", "projet 5", "projet 6"]
projetPlaces = [2 for i in range(len(projetName))]
print(projetPlaces)

#nbEleves = 18 + 42 + 2
# Attention méthode Yvon

[2, 2, 2, 2, 2, 2]


In [424]:
data = []
with open("choix.csv", "r") as f:
    choix = csv.reader(f)
    for line in choix:
        #cleaning data
        line[2] = line[1] + " " + line[2] 
        line = line[2::]
        print(line)
        data.append(list(line))
data = data[1::]

['Nom Prénom', 'Mes préférences\n [Projet 1]', 'Mes préférences\n [Projet 2]', 'Mes préférences\n [Projet 3]', 'Mes préférences\n [Projet 4]', 'Mes préférences\n [Projet 5]', 'Mes préférences\n [Projet 6]']
['eleve 9 123564', '1', '2', '3', '5', '6', '4']
['eleve 10 324516', '3', '2', '4', '5', '1', '6']
['eleve 1 123456', '1', '2', '3', '4', '5', '6']
['eleve 2 134562', '1', '3', '4', '5', '6', '2']
['eleve 3 234561', '2', '3', '4', '5', '6', '1']
['Eleve 4 345612', '3', '4', '5', '6', '1', '2']
['eleve 5 145623', '1', '4', '5', '6', '2', '3']
['eleve 6 156234', '1', '5', '6', '2', '3', '4']
['eleve 7 123564', '1', '2', '3', '5', '6', '4']
['eleve 8 234156', '2', '3', '4', '1', '5', '6']
['Boutaric  Clément ', '1', '2', '3', '4', '5', '6']
['Landais Joris', '6', '5', '4', '3', '2', '1']


In [425]:
preferenceE = deepcopy(data)
for i, line in enumerate(preferenceE):
    for j,elem in enumerate(line[1::]):
        preferenceE[i][j + 1] = int(elem)
for line in preferenceE:
    line = line.insert(1,1)
for line in preferenceE:
    line = line.insert(2, False)
for i,line in enumerate(preferenceE):
    line = line.insert(0,i)

print(preferenceE, type(preferenceE[0][6]))

#variable sur lse indices
iId = 0
iName = 1
iRank = 2
iAttributed = 3
iChoice = 4

[[0, 'eleve 9 123564', 1, False, 1, 2, 3, 5, 6, 4], [1, 'eleve 10 324516', 1, False, 3, 2, 4, 5, 1, 6], [2, 'eleve 1 123456', 1, False, 1, 2, 3, 4, 5, 6], [3, 'eleve 2 134562', 1, False, 1, 3, 4, 5, 6, 2], [4, 'eleve 3 234561', 1, False, 2, 3, 4, 5, 6, 1], [5, 'Eleve 4 345612', 1, False, 3, 4, 5, 6, 1, 2], [6, 'eleve 5 145623', 1, False, 1, 4, 5, 6, 2, 3], [7, 'eleve 6 156234', 1, False, 1, 5, 6, 2, 3, 4], [8, 'eleve 7 123564', 1, False, 1, 2, 3, 5, 6, 4], [9, 'eleve 8 234156', 1, False, 2, 3, 4, 1, 5, 6], [10, 'Boutaric  Clément ', 1, False, 1, 2, 3, 4, 5, 6], [11, 'Landais Joris', 1, False, 6, 5, 4, 3, 2, 1]] <class 'int'>


L'algo nécessite 3 tables:
* Table des préférences des élèves 
La structure est [id, Nom Prenom, rang du projet actuel, attribué, rang projet x]
* Table de préférence des projets
La structure est une liste, dont l'ordre des chiffres indique la préférence pour les élèves
* Table d'attribution
Chaque ligne est un projet, et chaque colonne indique l'élève qui y a été assigné, sans préférence d'ordre

In [426]:
#construction de la liste de préférence projet
preferenceP = []
for i in range(len(projetName)):
    ls = [i for i in range(len(preferenceE))]
    rd.shuffle(ls)
    preferenceP.append(ls)
print(preferenceP)

[[9, 8, 10, 3, 4, 0, 2, 11, 6, 5, 7, 1], [8, 6, 1, 10, 3, 7, 9, 5, 11, 4, 0, 2], [5, 0, 8, 11, 10, 4, 1, 2, 6, 3, 9, 7], [7, 5, 4, 11, 8, 2, 0, 1, 3, 9, 10, 6], [2, 10, 5, 4, 6, 9, 0, 1, 8, 11, 3, 7], [9, 10, 4, 2, 6, 7, 5, 0, 1, 11, 8, 3]]


In [427]:
#useful functions
def fullProject(lsAlloc, places):
    if len(lsAlloc) < places:
        return False
    else:
        return True

def projetActual(student, rank):
    #out projet number
    for i in range(iChoice, len(student)):
        if student[i] == rank:
            return i - iChoice

def stillUnallocated(prefE):
    #out: boolean if continue, and list of unallocated
    unallocated = []
    for line in prefE:
        if not line[iAttributed]:
            unallocated.append(line)
    if len(unallocated) == 0:
        return (False, unallocated)
    else:
        return True, unallocated

def leastPreferedStudent(allocProjet, prefP):
    """in: list of allocation for a projet and preference of this projet
    out: id of the least prefered student, index of least prefered student in allocProjet """
    maxPos = 0
    leastStudent = None
    for student in allocProjet:
        #find the position of preference of student
        position = prefP.index(student)
        if position > maxPos:
            maxPos = position
            leastStudent = student
    return leastStudent, allocProjet.index(leastStudent)

In [428]:
print("projetACtual", projetActual(preferenceE[1], preferenceE[1][iRank]))
print("stillUnallocated", stillUnallocated(preferenceE))
print("leastPreferedStudent", leastPreferedStudent([0, 3, 2], [0, 1, 2, 3, 4, 5]))

projetACtual 4
stillUnallocated (True, [[0, 'eleve 9 123564', 1, False, 1, 2, 3, 5, 6, 4], [1, 'eleve 10 324516', 1, False, 3, 2, 4, 5, 1, 6], [2, 'eleve 1 123456', 1, False, 1, 2, 3, 4, 5, 6], [3, 'eleve 2 134562', 1, False, 1, 3, 4, 5, 6, 2], [4, 'eleve 3 234561', 1, False, 2, 3, 4, 5, 6, 1], [5, 'Eleve 4 345612', 1, False, 3, 4, 5, 6, 1, 2], [6, 'eleve 5 145623', 1, False, 1, 4, 5, 6, 2, 3], [7, 'eleve 6 156234', 1, False, 1, 5, 6, 2, 3, 4], [8, 'eleve 7 123564', 1, False, 1, 2, 3, 5, 6, 4], [9, 'eleve 8 234156', 1, False, 2, 3, 4, 1, 5, 6], [10, 'Boutaric  Clément ', 1, False, 1, 2, 3, 4, 5, 6], [11, 'Landais Joris', 1, False, 6, 5, 4, 3, 2, 1]])
leastPreferedStudent (3, 1)


In [429]:
def galeShapley(prefE, prefP):
    allocation = [[] for i in range(len(prefP))]
    while stillUnallocated(prefE)[0]:
        for student in stillUnallocated(prefE)[1]:
            askProjet = projetActual(student, student[iRank])
            if not fullProject(allocation[askProjet], projetPlaces[askProjet]):
                allocation[askProjet].append(student[iId])
                student[iAttributed] = True
            else:
                #print(f"full for project {askProjet}, with student {student}, with actual allocation {allocation[askProjet]}")
                leastStudent, maxPos = leastPreferedStudent(allocation[askProjet], prefP[askProjet])
                if leastStudent == student[iId]:
                    student[iRank] += 1
                else:
                    allocation[askProjet][maxPos] = student[iId]
                    student[iAttributed] = True
                    prefE[leastStudent][iRank] +=1
                    prefE[leastStudent][iAttributed] = False
    return allocation

Vu que la préférence est aléatoire, on va faire tourner plusieurs fois l'algorithme pour déterminer la configuration la plus favorable. Le critère est le nombre de gens qui ont le rang le plus élevé. Si ça ne suffit pas à départager, on prendra en compte la moyenne.


In [430]:
def satisfaction(attribution, prefE):
    maxRank = 1
    nbmax = 0
    for i in range(len(attribution)):
        for j in range(len(attribution[i])):
            studentId = attribution[i][j]
            actualRank = prefE[studentId][iRank]
            if actualRank >  maxRank:
                maxRank = actualRank
                nbmax = 1
            elif actualRank == maxRank:
                nbmax += 1
                
    return maxRank, nbmax
    

In [431]:
allocation = galeShapley(preferenceE, preferenceP)
print(allocation)
print(preferenceE)
satisfaction(allocation, preferenceE)

[[8, 10], [3, 1], [2, 0], [9, 7], [11, 5], [4, 6]]
[[0, 'eleve 9 123564', 3, True, 1, 2, 3, 5, 6, 4], [1, 'eleve 10 324516', 2, True, 3, 2, 4, 5, 1, 6], [2, 'eleve 1 123456', 3, True, 1, 2, 3, 4, 5, 6], [3, 'eleve 2 134562', 3, True, 1, 3, 4, 5, 6, 2], [4, 'eleve 3 234561', 1, True, 2, 3, 4, 5, 6, 1], [5, 'Eleve 4 345612', 1, True, 3, 4, 5, 6, 1, 2], [6, 'eleve 5 145623', 3, True, 1, 4, 5, 6, 2, 3], [7, 'eleve 6 156234', 2, True, 1, 5, 6, 2, 3, 4], [8, 'eleve 7 123564', 1, True, 1, 2, 3, 5, 6, 4], [9, 'eleve 8 234156', 1, True, 2, 3, 4, 1, 5, 6], [10, 'Boutaric  Clément ', 1, True, 1, 2, 3, 4, 5, 6], [11, 'Landais Joris', 2, True, 6, 5, 4, 3, 2, 1]]


(3, 4)

In [432]:
maxSatisfaction = [len(projetName), len(preferenceE)]
maxAllocation = []

for i in range(100):
    #re initialize preferenceE et preferenceP
    preferenceE = deepcopy(data)
    for i, line in enumerate(preferenceE):
        for j,elem in enumerate(line[1::]):
            preferenceE[i][j + 1] = int(elem)
    for line in preferenceE:
        line = line.insert(1,1)
    for line in preferenceE:
        line = line.insert(2, False)
    for i,line in enumerate(preferenceE):
        line = line.insert(0,i)

    preferenceP = []
    for i in range(len(projetName)):
        ls = [i for i in range(len(preferenceE))]
        rd.shuffle(ls)
        preferenceP.append(ls)

    #choose preferred attribution
    actualAllocation = galeShapley(preferenceE, preferenceP)
    actualSatisfaction = satisfaction(allocation, preferenceE)

    if actualSatisfaction[0] < maxSatisfaction[0]:
        print(f"changed something with satisfaction actual {actualSatisfaction} and max {maxSatisfaction}")
        maXallocation = actualAllocation
        maxSatisfaction = actualSatisfaction
    elif actualSatisfaction[0] == maxSatisfaction[0]:
        if actualSatisfaction[1] < maxSatisfaction[1]:
            print(f"changed something after draw with satisfaction actual {actualSatisfaction} and max {maxSatisfaction}")
            maxAllocation = actualAllocation   
            maxSatisfaction = actualSatisfaction

print(maxAllocation, satisfaction(maxAllocation, preferenceE)) 

changed something with satisfaction actual (3, 4) and max [6, 12]
changed something after draw with satisfaction actual (3, 2) and max (3, 4)
[[6, 4], [8, 10], [0, 2], [9, 7], [1, 5], [3, 11]] (3, 2)


In [433]:
#re assign name to number
for i, projet in enumerate(maxAllocation):
    for j, student in enumerate(projet):
        maxAllocation[i][j] = preferenceE[student][iName]

for i in range(len(maxAllocation)):
    maxAllocation[i].insert(0, projetName[i])
print(maxAllocation)

[['projet coolos', 'eleve 5 145623', 'eleve 3 234561'], ['projet claqué', 'eleve 7 123564', 'Boutaric  Clément '], ['projet 3', 'eleve 9 123564', 'eleve 1 123456'], ['projet 4', 'eleve 8 234156', 'eleve 6 156234'], ['projet 5', 'eleve 10 324516', 'Eleve 4 345612'], ['projet 6', 'eleve 2 134562', 'Landais Joris']]


In [435]:
with open("result.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(maxAllocation)

Reste plus qu'à
- implémenter l'ordre aléatoire (mais est-ce vraiment utile, comme le nombre de frustré sera toujours le même)
- Rendre le résultat avec un csv et les noms